In [2]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import sklearn
import psycopg2 as psy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


The aim of this project is to identify clusters (that are meaningful, useful, or both that share common characteristics) of flight delays for FAA regulated airports in the US). Data includes name of airport, state, city, operational year (from 2004 to 2010), departures, arrivals, cancelations, numbers of delays and diversions, average delay times for a number of operational processes (such as taxi-out, taxi-in, airborne time, etc.). Please see detailed description of the features here: http://aspmhelp.faa.gov/index.php/APM:_Analysis:_Definitions_of_Variables


##  Part 1: Creating PostgreSQL Database

In [2]:
air_oper = pd.read_csv('/Users/Marina/Documents/GA Data Science/projects/projects-weekly/project-07/assets/Airport_operations.csv',)
air_canc = pd.read_csv('/Users/Marina/Documents/GA Data Science/projects/projects-weekly/project-07/assets/airport_cancellations.csv')
air_info =pd.read_csv('/Users/Marina/Documents/GA Data Science/projects/projects-weekly/project-07/assets/airports.csv')

In [3]:
air_oper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 15 columns):
airport                               841 non-null object
year                                  841 non-null int64
departures for metric computation     841 non-null int64
arrivals for metric computation       841 non-null int64
percent on-time gate departures       841 non-null float64
percent on-time airport departures    841 non-null float64
percent on-time gate arrivals         841 non-null float64
average_gate_departure_delay          841 non-null float64
average_taxi_out_time                 841 non-null float64
average taxi out delay                841 non-null float64
average airport departure delay       841 non-null float64
average airborne delay                841 non-null float64
average taxi in delay                 841 non-null float64
average block delay                   841 non-null float64
average gate arrival delay            841 non-null float64
dtypes: float64(1

In [4]:
### I am going to abbreviate my variables for each data set

air_oper.columns = ['airport', 'year','dept_comp','arriv_comp', 'ontime_gate_dept', 'ontine_air_dept','ontime_gate_arriv','gate_delay','taxi_out_time','taxi_out_delay', 'air_dept_delay','airborn_delay','taxi_in_delay','block_delay', 'gate_arrive_delay']

In [5]:
air_oper.head(5)

,airport,year,dept_comp,arriv_comp,ontime_gate_dept,ontine_air_dept,ontime_gate_arriv,gate_delay,taxi_out_time,taxi_out_delay,air_dept_delay,airborn_delay,taxi_in_delay,block_delay,gate_arrive_delay
0,ABQ,2004,53971,53818,0.8030,0.7809,0.7921,10.38,9.89,2.43,12.10,2.46,0.83,2.55,10.87
1,ABQ,2005,51829,51877,0.8140,0.7922,0.8001,9.60,9.79,2.29,11.20,2.26,0.89,2.34,10.24
2,ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
3,ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.40,12.95,2.19,1.29,3.06,12.71
4,ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48


In [6]:
air_canc.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 805 entries, 0 to 804
Data columns (total 6 columns):
Airport                    805 non-null object
Year                       805 non-null float64
Departure Cancellations    805 non-null float64
Arrival Cancellations      805 non-null float64
Departure Diversions       805 non-null float64
Arrival Diversions         805 non-null float64
dtypes: float64(5), object(1)
memory usage: 37.8+ KB


In [7]:
air_canc.columns = ['airport', 'year','dept_cancel','arriv_cancel', 'dept_diversions', 'arriv_diversions']

In [8]:
air_canc['year']=air_canc['year'].apply(lambda x: int(x))

In [9]:
air_canc.head(5)

,airport,year,dept_cancel,arriv_cancel,dept_diversions,arriv_diversions
0,ABQ,2004,242.0,235.0,71.0,46.0
1,ABQ,2005,221.0,190.0,61.0,33.0
2,ABQ,2006,392.0,329.0,71.0,124.0
3,ABQ,2007,366.0,304.0,107.0,45.0
4,ABQ,2008,333.0,300.0,79.0,42.0


In [10]:
air_canc['airport'].value_counts()

LAS    11
MHT    11
OAK    11
DCA    11
PVD    11
SEA    11
TPA    11
BWI    11
PHL    11
BUF    11
DAY    11
ONT    11
SAT    11
RDU    11
MEM    11
ISP    11
IAH    11
BUR    11
SWF    11
SJC    11
TUS    11
MKE    11
SMF    11
MSY    11
MIA    11
ATL    11
BOS    11
MDW    11
HNL    11
MSP    11
       ..
CLE    11
LGA    11
SNA    11
SFO    11
SDF    11
PHX    11
LAX    11
IND    11
ORD    11
OGG    11
RSW    11
PIT    11
OMA    11
MCO    11
SLC    11
DFW    11
IAD    11
SJU    11
ABQ    11
AUS    11
HOU    11
HPN    11
CLT    11
EWR    11
BNA    11
PBI    11
DAL    11
JFK    11
OXR     7
RFD     6
Name: airport, dtype: int64

In [11]:
air_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5167 entries, 0 to 5166
Data columns (total 13 columns):
Key                        5164 non-null float64
LocID                      5152 non-null object
AP_NAME                    5164 non-null object
ALIAS                      3498 non-null object
Facility Type              5164 non-null object
FAA REGION                 5164 non-null object
COUNTY                     5164 non-null object
CITY                       5164 non-null object
STATE                      5164 non-null object
AP Type                    5164 non-null object
Latitude                   5164 non-null float64
Longitude                  5164 non-null float64
Boundary Data Available    5164 non-null object
dtypes: float64(3), object(10)
memory usage: 524.8+ KB


In [35]:
air_info.columns = ['key', 'loc_id','ap_name','alias', 'fac_type', 'faa_region', 'county','city','state', 'ap_type', 'latitude', 'longitude', 'bound_data']

In [36]:
air_info.head(5)

,key,loc_id,ap_name,alias,fac_type,faa_region,county,city,state,ap_type,latitude,longitude,bound_data
0,3443.0,STX,HENRY E ROHLSEN,Henry E Rohlsen Int'l Airport,Airport,ASO,-VIRGIN ISLANDS-,CHRISTIANSTED,VI,Public Use,17.701556,-64.801722,Yes
1,5088.0,X64,PATILLAS,NaN,Airport,ASO,#NAME?,PATILLAS,PR,Public Use,17.982189,-66.019330,No
2,2886.0,PSE,MERCEDITA,Aeropuerto Mercedita,Airport,ASO,#NAME?,PONCE,PR,Public Use,18.008306,-66.563028,Yes
3,2879.0,VQS,ANTONIO RIVERA RODRIGUEZ,Aeropuerto Antonio Rivera Rodr�guez,Airport,ASO,#NAME?,ISLA DE VIEQUES,PR,Public Use,18.134811,-65.493617,Yes
4,2883.0,X63,HUMACAO,Aeropuerto Regional De Humacao,Airport,ASO,#NAME?,HUMACAO,PR,Public Use,18.138017,-65.800718,Yes


In [14]:
air_info['loc_id'].value_counts()

3A4    1
93F    1
7Y2    1
RHV    1
3B4    1
09I    1
09J    1
09K    1
09M    1
09N    1
8G3    1
09A    1
4I9    1
51K    1
51J    1
9W3    1
COU    1
51A    1
51D    1
W12    1
W13    1
09S    1
66S    1
HRL    1
93Y    1
53W    1
4P3    1
4P2    1
HRJ    1
80D    1
      ..
S84    1
4C4    1
S85    1
LAS    1
S87    1
S88    1
S89    1
7D8    1
FFA    1
O57    1
1Z1    1
HEE    1
HEI    1
HEF    1
HEG    1
I91    1
1DS    1
I93    1
I92    1
I99    1
5Y4    1
A28    1
A20    1
1Z9    1
8KA    1
A23    1
BJC    1
A26    1
61J    1
AGO    1
Name: loc_id, dtype: int64

#### Creating  PostgreSQL database

In [15]:
%reload_ext sql

In [16]:
from sqlalchemy import create_engine

In [21]:
engine = create_engine('postgresql://Marina@localhost:5432/air')

In [22]:
%sql postgresql://Marina@localhost:5432/air

u'Connected: Marina@air'

In [48]:
air_oper.to_sql('operations', engine,if_exists='replace')
air_canc.to_sql('cancelations', engine,if_exists='replace')
air_info.to_sql('information', engine,if_exists='replace')

In [49]:
%%sql

ALTER TABLE operations DROP column IF EXISTS index;

Done.


[]

In [50]:
%%sql

SELECT * FROM operations LIMIT 5;

5 rows affected.


airport,year,dept_comp,arriv_comp,ontime_gate_dept,ontine_air_dept,ontime_gate_arriv,gate_delay,taxi_out_time,taxi_out_delay,air_dept_delay,airborn_delay,taxi_in_delay,block_delay,gate_arrive_delay
ABQ,2004,53971,53818,0.803,0.7809,0.7921,10.38,9.89,2.43,12.1,2.46,0.83,2.55,10.87
ABQ,2005,51829,51877,0.814,0.7922,0.8001,9.6,9.79,2.29,11.2,2.26,0.89,2.34,10.24
ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.4,12.95,2.19,1.29,3.06,12.71
ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48


In [51]:
%%sql

ALTER TABLE cancelations DROP column IF EXISTS index;

Done.


[]

In [52]:
%%sql

SELECT * FROM cancelations LIMIT 5;

5 rows affected.


airport,year,dept_cancel,arriv_cancel,dept_diversions,arriv_diversions
ABQ,2004,242.0,235.0,71.0,46.0
ABQ,2005,221.0,190.0,61.0,33.0
ABQ,2006,392.0,329.0,71.0,124.0
ABQ,2007,366.0,304.0,107.0,45.0
ABQ,2008,333.0,300.0,79.0,42.0


In [53]:
%%sql

ALTER TABLE information DROP column IF EXISTS index;

Done.


[]

In [54]:
%%sql

SELECT * FROM information LIMIT 5;

5 rows affected.


key,loc_id,ap_name,alias,fac_type,faa_region,county,city,state,ap_type,latitude,longitude,bound_data
3443.0,STX,HENRY E ROHLSEN,Henry E Rohlsen Int'l Airport,Airport,ASO,-VIRGIN ISLANDS-,CHRISTIANSTED,VI,Public Use,17.701556,-64.801722,Yes
5088.0,X64,PATILLAS,None,Airport,ASO,#NAME?,PATILLAS,PR,Public Use,17.982189,-66.01933,No
2886.0,PSE,MERCEDITA,Aeropuerto Mercedita,Airport,ASO,#NAME?,PONCE,PR,Public Use,18.008306,-66.563028,Yes
2879.0,VQS,ANTONIO RIVERA RODRIGUEZ,Aeropuerto Antonio Rivera Rodr�guez,Airport,ASO,#NAME?,ISLA DE VIEQUES,PR,Public Use,18.134811,-65.493617,Yes
2883.0,X63,HUMACAO,Aeropuerto Regional De Humacao,Airport,ASO,#NAME?,HUMACAO,PR,Public Use,18.138017,-65.800718,Yes


Load our csv files into tables

In [55]:
%%sql

ALTER TABLE operations RENAME airport TO airport2
    


Done.


[]

In [56]:
%%sql
ALTER TABLE operations RENAME year TO year2

Done.


[]

Join airport_cancellations.csv and airports.csv into one table

In [57]:
%%sql

SELECT * FROM operations

841 rows affected.


airport2,year2,dept_comp,arriv_comp,ontime_gate_dept,ontine_air_dept,ontime_gate_arriv,gate_delay,taxi_out_time,taxi_out_delay,air_dept_delay,airborn_delay,taxi_in_delay,block_delay,gate_arrive_delay
ABQ,2004,53971,53818,0.803,0.7809,0.7921,10.38,9.89,2.43,12.1,2.46,0.83,2.55,10.87
ABQ,2005,51829,51877,0.814,0.7922,0.8001,9.6,9.79,2.29,11.2,2.26,0.89,2.34,10.24
ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.4,12.95,2.19,1.29,3.06,12.71
ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48
ABQ,2009,42550,42704,0.8341,0.8147,0.8224,9.24,9.88,1.89,10.45,1.74,0.92,1.98,9.27
ABQ,2010,42302,42169,0.8144,0.7907,0.8047,10.25,9.95,2.05,11.58,1.66,0.95,1.9,10.23
ABQ,2011,42283,42297,0.8072,0.7758,0.8022,10.81,10.5,2.51,12.57,1.58,0.84,1.87,10.63
ABQ,2012,39105,39107,0.8219,0.794,0.8177,10.55,10.27,2.32,12.11,1.41,0.98,1.8,9.73
ABQ,2013,35277,34989,0.771,0.7402,0.7554,12.56,10.77,2.64,14.28,1.6,0.92,2.35,12.76


In [64]:
%%sql


SELECT * INTO table4 FROM cancelations INNER JOIN operations
ON cancelations.airport = operations.airport2 AND cancelations.year = operations.year2




799 rows affected.


[]

In [66]:
%%sql
SELECT * FROM table4

799 rows affected.


airport,year,dept_cancel,arriv_cancel,dept_diversions,arriv_diversions,airport2,year2,dept_comp,arriv_comp,ontime_gate_dept,ontine_air_dept,ontime_gate_arriv,gate_delay,taxi_out_time,taxi_out_delay,air_dept_delay,airborn_delay,taxi_in_delay,block_delay,gate_arrive_delay
ABQ,2004,242.0,235.0,71.0,46.0,ABQ,2004,53971,53818,0.803,0.7809,0.7921,10.38,9.89,2.43,12.1,2.46,0.83,2.55,10.87
ABQ,2005,221.0,190.0,61.0,33.0,ABQ,2005,51829,51877,0.814,0.7922,0.8001,9.6,9.79,2.29,11.2,2.26,0.89,2.34,10.24
ABQ,2006,392.0,329.0,71.0,124.0,ABQ,2006,49682,51199,0.7983,0.7756,0.7746,10.84,9.89,2.16,12.33,2.12,0.84,2.66,11.82
ABQ,2007,366.0,304.0,107.0,45.0,ABQ,2007,53255,53611,0.8005,0.7704,0.7647,11.29,10.34,2.4,12.95,2.19,1.29,3.06,12.71
ABQ,2008,333.0,300.0,79.0,42.0,ABQ,2008,49589,49512,0.8103,0.7844,0.7875,10.79,10.41,2.41,12.32,1.82,1.03,2.79,11.48
ABQ,2009,192.0,162.0,74.0,26.0,ABQ,2009,42550,42704,0.8341,0.8147,0.8224,9.24,9.88,1.89,10.45,1.74,0.92,1.98,9.27
ABQ,2010,222.0,177.0,71.0,50.0,ABQ,2010,42302,42169,0.8144,0.7907,0.8047,10.25,9.95,2.05,11.58,1.66,0.95,1.9,10.23
ABQ,2011,345.0,294.0,77.0,61.0,ABQ,2011,42283,42297,0.8072,0.7758,0.8022,10.81,10.5,2.51,12.57,1.58,0.84,1.87,10.63
ABQ,2012,237.0,181.0,65.0,60.0,ABQ,2012,39105,39107,0.8219,0.794,0.8177,10.55,10.27,2.32,12.11,1.41,0.98,1.8,9.73
ABQ,2013,232.0,212.0,62.0,46.0,ABQ,2013,35277,34989,0.771,0.7402,0.7554,12.56,10.77,2.64,14.28,1.6,0.92,2.35,12.76


In [67]:

df = %sql SELECT * FROM table4 INNER JOIN information ON table4.airport = information.loc_id

df = df.DataFrame()
df.head(2)

799 rows affected.


,airport,year,dept_cancel,arriv_cancel,dept_diversions,arriv_diversions,airport2,year2,dept_comp,arriv_comp,...,alias,fac_type,faa_region,county,city,state,ap_type,latitude,longitude,bound_data
0,ABQ,2004,242.0,235.0,71.0,46.0,ABQ,2004,53971,53818,...,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,Federalized/Commercial,35.040194,-106.609194,Yes
1,ABQ,2005,221.0,190.0,61.0,33.0,ABQ,2005,51829,51877,...,Albuquerque Int'l Sunport,Airport,ASW,BERNALILLO,ALBUQUERQUE,NM,Federalized/Commercial,35.040194,-106.609194,Yes


Query the database for our intial data

In [68]:
df.to_csv(path_or_buf="/Users/Marina/Documents/GA Data Science/projects/projects-weekly/project-07/data_sql.csv", index=False, encoding='utf-8')